In [1]:
#! pip install geemap

In [1]:
import ee
import geemap

In [2]:
# If you are working with a shape file stored in google drive remove the quotes (""" """)

"""
#connect with drive
from google.colab import drive
drive.mount('/content/drive')
"""

"\n#connect with drive\nfrom google.colab import drive\ndrive.mount('/content/drive')\n"

In [3]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWj_o9Tz6KhKTWPcrMdhCOhuVovdmaFgbFe56WZ1h4NOwlOoovs4CuY

Successfully saved authorization token.


In [4]:
Map = geemap.Map()
#Map

In [5]:
# https://sites.google.com/site/seriescol/shapes
shape = ee.FeatureCollection('users/dsrestrepo/mpio')

# Cauca = shape.filter(ee.Filter.eq('DPTO_CCDGO', 19))
# region = shape.filter(ee.Filter.eq('MPIO_CCNCT	', 19001))
# Medellin = ee.FeatureCollection('users/dsrestrepo/Medellin')

region = shape.filter(ee.Filter.eq('MPIOS', '05001'))

Map.addLayer(region, {}, "Region")

In [6]:
# Define the range of years
initial_year = 2013
end_year = 2020
years = ee.List.sequence(initial_year, end_year)
# Define the range of months:
months = ee.List.sequence(1,12)
years.getInfo()
#months.getInfo()

[2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020]

# Get number of images each month

In [7]:
def images_per_month(year, month):
    
    start_date = ee.Date.fromYMD(year, month, 1)
    end_date = start_date.advance(1, "month")
    # https://developers.google.com/earth-engine/datasets/catalog/landsat-8
    #collection = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
        .filterDate(start_date, end_date) \
        .filterBounds(region)
    print(f'the images in month {month} of year {year} are {collection.size().getInfo()}')
    # Take month and year of month with 0 images
    if collection.size().getInfo() == 0:
        print('is zero')
        return year, month

In [8]:
months_0_images = []
# See images by month
for year in years.getInfo():
    for month in months.getInfo():
        months_0_images.append(images_per_month(year, month))
months_0_images = list(filter(None, months_0_images))

the images in month 1 of year 2013 are 0
is zero
the images in month 2 of year 2013 are 0
is zero
the images in month 3 of year 2013 are 0
is zero
the images in month 4 of year 2013 are 2
the images in month 5 of year 2013 are 2
the images in month 6 of year 2013 are 3
the images in month 7 of year 2013 are 1
the images in month 8 of year 2013 are 4
the images in month 9 of year 2013 are 4
the images in month 10 of year 2013 are 4
the images in month 11 of year 2013 are 1
the images in month 12 of year 2013 are 2
the images in month 1 of year 2014 are 4
the images in month 2 of year 2014 are 3
the images in month 3 of year 2014 are 3
the images in month 4 of year 2014 are 2
the images in month 5 of year 2014 are 2
the images in month 6 of year 2014 are 4
the images in month 7 of year 2014 are 2
the images in month 8 of year 2014 are 4
the images in month 9 of year 2014 are 3
the images in month 10 of year 2014 are 4
the images in month 11 of year 2014 are 4
the images in month 12 of ye

In [9]:
months_0_images

[(2013, 1), (2013, 2), (2013, 3), (2018, 9)]

# Create one image per month with as few clouds as possible

In [10]:
def monthly_image(year, month):
    
    # Define Start Date
    start_date = ee.Date.fromYMD(year, month, 1)
   
    # If no images in that month (2018 / 9)
    if ((year, month) in months_0_images):
        print('in')
        # Take more days before month starts
        end_date = start_date.advance(45, "days")
        #end_date = start_date.advance(70, "days")
        # Take more days after month ends
        start_date = start_date.advance(-15, "days")
        #start_date = start_date.advance(-40, "days")
    else:
        # ends one month after
        end_date = start_date.advance(1, "month")
    
    # Get the images for the month in the region
    collection = ee.ImageCollection('LANDSAT/LC08/C01/T1') \
        .filterDate(start_date, end_date) \
        .filterBounds(region)
    
    # https://developers.google.com/earth-engine/apidocs/ee-algorithms-landsat-simplecomposite
    image =  ee.Algorithms.Landsat.simpleComposite(collection,
                                                   percentile=20,
                                                   cloudScoreRange=5,
                                                   asFloat=True).clipToCollection(region)
    
    return image

# Increase image resolution (30m / pixel -> 15m / pixel)

In [11]:
# Pan sharpening Image
# https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0#qt-news_science_products
# B8: Panchromatic (15 m) resolution
def pan_sharpening_image(image):
    # Convert the RGB bands to the HSV color space.
    hsv = image.select(['B4', 'B3', 'B2']).rgbToHsv()
    sharpened = ee.Image.cat([hsv.select('hue'),
                             hsv.select('saturation'),
                             image.select('B8')]).hsvToRgb()
    
    return sharpened

# Get image dimensions

In [12]:
# Take a random image of dataset
test_year = 2014
test_month = 9
image = ee.Image(monthly_image(test_year, test_month))
# Increese image resolution 15 m/pixel
sharpened = pan_sharpening_image(image)

In [13]:
import numpy as np
# Mask image
image = sharpened.clip(region).unmask()

#https://gis.stackexchange.com/questions/350771/earth-engine-simplest-way-to-move-from-ee-image-to-array-for-use-in-sklearn/351177#351177
# Reproject image from 15m/px -> 120m/px due to the maximum number of bits allowed in the array.
# We select just one band a reference (red)
image = image.reproject(crs = image.select('red').projection(), scale= 120)
# Sample image as rectangle
band_arrs = image.sampleRectangle(region=region.geometry(), defaultValue= 0.0)
# take red band from array
band_arr_red = band_arrs.get('red')
# Array to numpy
np_arr_red = np.array(band_arr_red.getInfo())
# Shape of image will be aprox:
height = (8 * np_arr_red.shape[0]) - 8
width = (8 * np_arr_red.shape[1]) - 8
print(f'the dimensions of the image will be height: {height} and width: {width}')

the dimensions of the image will be height: 1560 and width: 2112


# Download RGB Image

In [14]:
# https://www.usgs.gov/faqs/what-are-band-designations-landsat-satellites?qt-news_science_products=0#qt-news_science_products
# B4: Red, B3: Green, B2: Blue
vis_params = {'bands': ['B4',  'B3',  'B2'],
              'max': 0.25,}
vis_params_sarped = {'bands': ['red', 'green', 'blue'],
                     'max': 0.25,}

In [78]:
from functools import partial

# Landsat Launch: 11 February 2013
# Data from April 2013
months_2013 = ee.List.sequence(4,12)

# For each year
for year in years.getInfo():
    
    # Create a new empty list of images
    images = ee.List([])
    if (year < 2013 or year > 2020):
        print('No images for that date')
        break
    # 2013 Starts late
    if year == 2013:
        # Bound monthly_image function with the year
        bound_monthly_image = partial(monthly_image, year)
        # Apply function to each month
        for month in months_2013.getInfo():
            # Get a single image each month
            images = images.add(bound_monthly_image(month))     
    else:
        # Bound monthly_image function with the year
        bound_monthly_image = partial(monthly_image, year)
        # Apply function to each month
        for month in months.getInfo():
            images = images.add(bound_monthly_image(month))   
            
    for index in range(0, 12):
        # Ignore the last 3 index from 2013 because data starts in april
        if (year == 2013 and index > 8):
            continue
        
        # Take form ee list the image
        image = ee.Image(images.get(index))
        
        # Name
        layer_name = "Image " + str(year) + "_" + str(index + 1)
        # 2013 starts in april
        if (year == 2013):
            layer_name = "Image " + str(year) + "_" + str(index + 4)
        
        
        # Pan Sharpening
        sharpened = pan_sharpening_image(image)
        
        # Download:
        ## To PC
        image_unmask = image.clip(region).unmask()
        geemap.get_image_thumbnail(image_unmask, 'Images/'+ layer_name + '.png', vis_params, dimensions=(width ,height), format='png', region=region.geometry())
        
        sharpened_unmask = sharpened.clip(region).unmask()
        geemap.get_image_thumbnail(sharpened_unmask, 'Images/'+ layer_name + '_sharpend_' + '.png', vis_params_sarped, dimensions=(width ,height), format='png', region=region.geometry())
        
        ## To Drive
        # Export the image, specifying scale and region.
        # https://developers.google.com/earth-engine/guides/reducers_reduce_region
        #geemap.ee_export_image_to_drive(sharpened, description= layer_name + '_sharpend', 
        #                                folder='GEE/Medellin', 
        #                                region=region.geometry(), 
        #                                scale=15)
                
        task = ee.batch.Export.image.toDrive(**{
            'image': sharpened,
            'description': layer_name + '_sharpend',
            'folder':'GEE/Medellin',
            'scale': 15,
            'region': region.geometry()
        })
        task.start()


in


In [81]:
task.status()

{'state': 'READY',
 'description': 'Image 2020_12_sharpend',
 'creation_timestamp_ms': 1628137333803,
 'update_timestamp_ms': 1628137333803,
 'start_timestamp_ms': 0,
 'task_type': 'EXPORT_IMAGE',
 'id': 'N5HPTIIXKTZXB66V6UVRBK4N',
 'name': 'projects/earthengine-legacy/operations/N5HPTIIXKTZXB66V6UVRBK4N'}

# See an example of the images

In [15]:
from functools import partial

# For the example just use some months of 2013
initial_year = 2013
end_year = 2013
years = ee.List.sequence(initial_year, end_year)


months_2013 = ee.List.sequence(4,12)

# For each year
for year in years.getInfo():
    
    # Create a new empty list of images
    images = ee.List([])
    if (year < 2013 or year > 2020):
        print('No images for that date')
        break
    # 2013 Starts late
    if year == 2013:
        # Bound monthly_image function with the year
        bound_monthly_image = partial(monthly_image, year)
        # Apply function to each month
        for month in months_2013.getInfo():
            # Get a single image each month
            images = images.add(bound_monthly_image(month))     
    else:
        # Bound monthly_image function with the year
        bound_monthly_image = partial(monthly_image, year)
        # Apply function to each month
        for month in months.getInfo():
            images = images.add(bound_monthly_image(month))   
            
    for index in range(0, 12):
        # Ignore the last 3 index from 2013 because data starts in april
        if (year == 2013 and index > 8):
            continue
        
        # Take form ee list the image
        image = ee.Image(images.get(index))
        
        # Name
        layer_name = "Image " + str(year) + "_" + str(index + 1)
        # 2013 starts in april
        if (year == 2013):
            layer_name = "Image " + str(year) + "_" + str(index + 4)
        
        
        # Pan Sharpening
        sharpened = pan_sharpening_image(image)
        
        sharpened_unmask = sharpened.clip(region).unmask()

        
        Map.addLayer(image, vis_params, layer_name)
        Map.addLayer(sharpened, vis_params_sarped, layer_name + " sharpened")
        

Map.center_object(region, 13)
Map

Map(center=[6.26500567846621, -75.63644335353737], controls=(WidgetControl(options=['position', 'transparent_b…